In [23]:
%%capture
!pip install -r https://raw.githubusercontent.com/jpcosta90/once-learning/refs/heads/main/scripts/requirements.txt

In [24]:
!llama model list

+----------------------------------+------------------------------------------+----------------+
| Model Descriptor                 | Hugging Face Repo                        | Context Length |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-8B                      | meta-llama/Llama-3.1-8B                  | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-70B                     | meta-llama/Llama-3.1-70B                 | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-405B:bf16-mp8           | meta-llama/Llama-3.1-405B                | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-405B                    | meta-llama/Llama-3.1-405B-FP8            | 128K           |
+-----------------------------

In [3]:
from huggingface_hub import snapshot_download
import os

# Get the token from the environment variable
hf_token = os.getenv("HF_TOKEN")

In [29]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [30]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    load_in_4bit=True,
)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [50]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jpcosta90/once-learning/refs/heads/main/data/one-page-document/one_page_document_comparisons.csv')
df = df[['file1','file2','same_pattern']]


In [51]:
file_name = df['file1'][0]
file_name

'pattern00/gnmm0220.tif'

In [48]:
file_name = 'pattern00/gnmm0220.tif'

In [52]:
%%time

url = f'https://raw.githubusercontent.com/jpcosta90/once-learning/main/files/{file_name}'
image = Image.open(requests.get(url, stream=True).raw)
# image = Image.open('/content/drive/MyDrive/Once_Learning_Docs/padrao/00001486.jpg')

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Explique a imagem em português: "}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=100)
print(processor.decode(output[0]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>Explique a imagem em português: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

A imagem é uma cópia de uma fatura de uma empresa, datada de setembro de 2004, emitida pela PK Noonan & Associates, LLC, localizada em Richmond, VA 23233. A fatura inclui os seguintes detalhes:

*   **Fatura**
    *   **Número de Invoice**: 8500008833
    *   **Data**: Setembro de 2004
    *   **Atenção**: Valerie A.
CPU times: user 18.9 s, sys: 1.13 s, total: 20 s
Wall time: 20.6 s
